In this short notebook, an H2O AutoML workflow is exemplified. 

In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.261-b12, mixed mode, sharing)
  Starting server from C:\Users\Renzo\AppData\Local\Programs\Python\Python38-32\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Renzo\AppData\Local\Temp\tmpjih69mje
  JVM stdout: C:\Users\Renzo\AppData\Local\Temp\tmpjih69mje\h2o_Renzo_started_from_python.out
  JVM stderr: C:\Users\Renzo\AppData\Local\Temp\tmpjih69mje\h2o_Renzo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,America/Denver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_Renzo_iu41k1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
#Load data into cluster
filename = "https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/product_backorders.csv"

df = h2o.import_file(filename)

df.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
1.11312e+06,0,8,1,6,6,6,0,4,9,12,0,No,1,0.9,0.89,0,No,No,No,Yes,No,Yes
1.11327e+06,0,8,0,2,3,4,1,2,3,3,0,No,0,0.96,0.97,0,No,No,No,Yes,No,Yes
1.11387e+06,20,2,0,45,99,153,16,42,80,111,10,No,0,0.81,0.88,0,No,No,No,Yes,No,Yes
1.11422e+06,0,8,0,9,14,21,5,17,36,43,0,No,0,0.96,0.98,0,No,No,No,Yes,No,Yes
1.11482e+06,0,12,0,31,31,31,7,15,33,47,2,No,3,0.98,0.98,0,No,No,No,Yes,No,Yes
1.11545e+06,55,8,0,216,360,492,30,108,275,340,51,No,0,0,0,0,No,No,Yes,Yes,No,Yes
1.11562e+06,-34,8,0,120,240,240,83,122,144,165,33,No,0,1,0.97,34,No,No,No,Yes,No,Yes
1.11645e+06,4,9,0,43,67,115,5,22,40,58,4,No,0,0.69,0.68,0,No,No,No,Yes,No,Yes
1.11683e+06,2,8,0,4,6,9,1,5,6,9,2,No,0,1,0.95,0,No,No,No,Yes,No,Yes
1.11687e+06,-7,8,0,56,96,112,13,30,56,76,0,No,0,0.97,0.92,7,No,No,No,Yes,No,Yes


In [3]:
#Splitting the dataset
splits = df.split_frame(ratios=[0.7], seed = 1)
train = splits[0]
test = splits[1]

y = "went_on_backorder"
x = df.columns
x.remove("sku")
x.remove(y)

#Run AutoML
aml = H2OAutoML(max_runtime_secs = 240, seed = 1)
aml.train(x = x, y = y, training_frame = train)







AutoML progress: |
21:48:23.532: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [4]:
lb = aml.leaderboard
lb.head(rows = lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200916_214823,0.947576,0.184902,0.739018,0.165098,0.230493,0.0531271
StackedEnsemble_BestOfFamily_AutoML_20200916_214823,0.947003,0.185546,0.738859,0.159836,0.230717,0.0532305
GBM_4_AutoML_20200916_214823,0.946255,0.177204,0.731942,0.158332,0.228648,0.05228
GBM_3_AutoML_20200916_214823,0.943897,0.180116,0.724003,0.159587,0.229858,0.0528346
GBM_2_AutoML_20200916_214823,0.941305,0.18557,0.713645,0.154123,0.233476,0.054511
GBM_5_AutoML_20200916_214823,0.93877,0.192216,0.705799,0.153604,0.238581,0.0569209
XRT_1_AutoML_20200916_214823,0.938658,0.206046,0.7154,0.166387,0.244745,0.0599
GBM_1_AutoML_20200916_214823,0.937308,0.19248,0.701135,0.150656,0.238145,0.0567131
DRF_1_AutoML_20200916_214823,0.930056,0.222322,0.679189,0.190602,0.255731,0.0653982
GBM_grid__1_AutoML_20200916_214823_model_1,0.889245,0.257196,0.550991,0.212981,0.27568,0.0759997
